In [1]:
!pip install --upgrade coremltools

     |████████████████████████████████| 3.4MB 12.9MB/s 
  Created wheel for attr: filename=attr-0.3.1-cp37-none-any.whl size=2458 sha256=2528d1b4cbcb289382d0c0ec3b7abce7f01753d1717cf1f0204a271383459e00
  Stored in directory: /root/.cache/pip/wheels/f0/96/9b/1f8892a707d17095b5a6eab0275da9d39e68e03a26aee2e726
Successfully built attr


In [2]:
pip install torch torchvision torchaudio

     |████████████████████████████████| 1.9MB 14.3MB/s 


In [3]:
import torch
import torchvision
import os

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

path = '/content/gdrive/MyDrive/Facial-Expression-Recognition.Pytorch-master/'

Mounted at /content/gdrive


In [5]:
#model = torch.load(path)
#example_input = torch.rand(1, 48, 48) 
#traced_model = torch.jit.trace(model, example_input)


In [6]:
os.chdir(path)
os.getcwd()
from models import *

In [7]:
net = VGG('VGG19')
checkpoint = torch.load('model.t7')
net.load_state_dict(checkpoint['net'])
net.eval()


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [8]:
from PIL import Image
from google.colab import drive
drive.mount('/content/gdrive')
import os

import transforms as transforms
path = '/content/gdrive/MyDrive/Facial-Expression-Recognition.Pytorch-master/'
os.chdir(path)

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from torch.autograd import Variable

import transforms as transforms
from skimage import io
from skimage.transform import resize
from models import *

cut_size = 48

transform_test = transforms.Compose([
    transforms.TenCrop(cut_size),
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
])

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

raw_img = io.imread('sadgirl.jpg')
gray = rgb2gray(raw_img)
gray = resize(gray, (48,48), mode='symmetric').astype(np.uint8)

img = gray[:, :, np.newaxis]

img = np.concatenate((img, img, img), axis=2)
img = Image.fromarray(img)
inputs = transform_test(img)

class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

net.eval()

ncrops, c, h, w = np.shape(inputs)

inputs = inputs.view(-1, c, h, w)
#inputs = inputs.cuda()
inputs = Variable(inputs, volatile=True)
#print(inputs)
example_input = inputs
outputs = net(inputs)

outputs_avg = outputs.view(ncrops, -1).mean(0)  # avg over crops
print(outputs_avg)

score = F.softmax(outputs_avg)
_, predicted = torch.max(outputs_avg.data, 0)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


tensor([ 0.2495, -1.5129,  1.0377, -0.6874,  0.7285, -0.0569,  0.3438],
       grad_fn=<MeanBackward1>)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [9]:

# Trace with random data
example_input = torch.rand(1,3, 48, 48) # after test, will get 'size mismatch' error message with size 256x256
model = torch.jit.trace(net, example_input)
emo_idx2label = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

import coremltools as ct
#Convert to Core ML using the Unified Conversion API
# res_model = ct.convert(
#     model,
#     inputs=[ct.TensorType(name="image", shape=example_input.shape)],
#     classifier_config=ct.ClassifierConfig(emo_idx2label)
# )
res_model = ct.convert(
    model,
    inputs=[ct.ImageType(name="image", shape=(example_input.shape), scale=1/255.0)],
    classifier_config=ct.ClassifierConfig(emo_idx2label)
)
# res_model = ct.convert(
#     model,
#     inputs=[ct.TensorType(name="input", shape=(example_input.shape))],
#     classifier_config=ct.ClassifierConfig(emo_idx2label)
# )



Translating MIL ==> MLModel Ops: 100%|██████████| 246/246 [00:02<00:00, 98.43 ops/s]


In [10]:
res_model.save("myvgg5.mlmodel")

In [11]:
import coremltools
import coremltools.proto.FeatureTypes_pb2 as ft 

spec = coremltools.utils.load_spec("myvgg5.mlmodel")
print(spec.description)
input = spec.description.input[0]
input.type.imageType.colorSpace = ft.ImageFeatureType.GRAYSCALE

coremltools.utils.save_spec(spec, "YourNewModel.mlmodel")
print(spec.description)

input {
  name: "image"
  type {
    imageType {
      width: 48
      height: 48
      colorSpace: RGB
    }
  }
}
output {
  name: "260"
  type {
    dictionaryType {
      stringKeyType {
      }
    }
  }
}
output {
  name: "classLabel"
  type {
    stringType {
    }
  }
}
predictedFeatureName: "classLabel"
predictedProbabilitiesName: "260"
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "torch==1.8.1+cu101"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "4.1"
  }
}

input {
  name: "image"
  type {
    imageType {
      width: 48
      height: 48
      colorSpace: GRAYSCALE
    }
  }
}
output {
  name: "260"
  type {
    dictionaryType {
      stringKeyType {
      }
    }
  }
}
output {
  name: "classLabel"
  type {
    stringType {
    }
  }
}
predictedFeatureName: "classLabel"
predictedProbabilitiesName: "260"
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "torch==1.8.1+c